In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import pytz

import functions_PyMetatrader5 as fnmt5

In [14]:
indice = pd.read_csv('files\\Unemployment Rate - United States.csv', index_col='DateTime')
indice.drop('Revised', axis=1, inplace=True)
indice.index = pd.to_datetime(indice.index)
indice.tail()

,Actual,Consensus,Previous
DateTime,,,
2007-07-06 12:30:00,4.5,4.5,4.5
2007-06-01 12:30:00,4.5,4.5,4.5
2007-05-04 12:30:00,4.5,4.5,4.4
2007-04-06 12:30:00,4.4,4.6,4.5
2007-03-09 13:30:00,4.5,4.6,NaN


In [15]:
np.random.seed(545)
escenarios = np.random.choice([i for i in range(len(indice))], 5)
asset = 'USDCHF'
dates = [[indice.iloc[i].name, indice.iloc[i].name + datetime.timedelta(hours=0.5)] for i in escenarios ]
escenarios

array([ 44,   5,  39, 129,  38])

In [16]:
# local_exe = 'C:\\Program Files\\MetaTrader 5\\terminal64.exe'
local_exe = 'C:\\Archivos de programa\\MetaTrader 5 Terminal\\terminal64.exe'

mt5_acc = 5383442
mt5_pass = "44GxKTtf"
mt5_client = fnmt5.f_init_login(mt5_acc, mt5_pass, local_exe)




 ++++ init_login succeded, message =  True


In [20]:
def is_dst(when):
    return when.dst() != timedelta(0)
def validadiciones(df_index: pd.DataFrame, mt5_client, symbol):
    # semilla
    np.random.seed(545)
    # 5 fechas aleatorioas
    escenarios = np.random.choice([i for i in range(len(df_index))], 5)
    # Agregamos media hora a todas las fechas
    tf = datetime.timedelta(hours=0.5)
    # Ajusta para daylight savings
    dates = [[df_index.iloc[i].name, df_index.iloc[i].name + tf] for i in escenarios]

    # descarga de precios
    prices = [fnmt5.f_hist_prices(mt5_client, [symbol], 'M1', dates[i][0], dates[i][-1]).get(asset) for i in range(len(dates))]
    
    # separacion y columnas de tiempo
    escenario1 = prices[0]
    escenario1['time'] = [datetime.datetime.utcfromtimestamp(times) for times in escenario1['time']]
    
    escenario2 = prices[1]
    escenario2['time'] = [datetime.datetime.utcfromtimestamp(times) for times in escenario2['time']]

    escenario3 = prices[2]
    escenario3['time'] = [datetime.datetime.utcfromtimestamp(times) for times in escenario3['time']]

    
    escenario4 = prices[3]
    escenario4['time'] = [datetime.datetime.utcfromtimestamp(times) for times in escenario4['time']]

    
    escenario5 = prices[4]
    escenario5['time'] = [datetime.datetime.utcfromtimestamp(times) for times in escenario5['time']]

    
    return escenario1, escenario2, escenario3, escenario4, escenario5

In [19]:
def escenarios(fila):
    
    if fila[0]>=fila[1]>=fila[2]:
        escenario = 'A'
    elif fila[0]>=fila[1]<fila[2]:
        escenario = 'B'
    elif fila[0]<fila[1]>=fila[2]:
        escenario = 'C'
    else:
        escenario = 'D'
    return escenario

indice.iloc[-1]['Previous'] = indice.iloc[-2]['Actual']

In [72]:
asset = 'USDCHF'

df_escenarios = pd.DataFrame(index=indice.index)
df_escenarios['Escenario'] = [escenarios(indice.iloc[i].to_list()) for i in range(len(indice))]

tf = datetime.timedelta(hours=0.5)
dst_adjust = datetime.timedelta(hours=1)

dates = [df_escenarios.iloc[i].name - tf + dst_adjust for i in range(len(df_escenarios)) ]

prices = [fnmt5.f_hist_prices_from(mt5_client, [asset], 'M1', dates[i], 61).get(asset) for i in range(len(dates))]


In [73]:
pip_size = 10000
direccion = []
pip_alcistas = []
pip_bajistas = []
vol = []
for i in range(len(df_escenarios)):
    # Para cada uno de los precios
    df = prices[i]
    # Convertir columnas de tiempo
    df['time'] = [datetime.datetime.utcfromtimestamp(times) for times in df['time']]
    # Direccion
    direc = df['close'].iloc[-1] - df['open'].iloc[30]
    # Validaciòn de direcciòn
    if direc<=0:
        direccion.append(1)
    else:
        direccion.append(-1)
    # Pips alcistas
    high = (df['high'].iloc[30:-1].max() - df['open'].iloc[30])*pip_size
    # Pips bajistas
    low = (df['open'].iloc[30] - df['low'].iloc[30:-1].min())*pip_size
    # volatilidad
    vola = (df['high'].max() - df['low'].min())*pip_size
    
    pip_alcistas.append(high)
    pip_bajistas.append(low)
    vol.append(vola)
    
df_escenarios['Direccion'] = direccion
df_escenarios['pip_alcistas'] = pip_alcistas
df_escenarios['pips_bajistas'] = pip_bajistas
df_escenarios['volatilidad'] = vol

display(df_escenarios.head(len(df_escenarios)))

,Escenario,Direccion,pip_alcistas,pips_bajistas,volatilidad
DateTime,,,,,
2020-02-07 13:30:00,A,-1,5.6,1.0,12.7
2020-01-10 13:30:00,A,1,0.7,5.0,11.3
2019-12-06 13:30:00,C,-1,5.2,0.0,5.2
2019-11-01 12:30:00,A,1,0.9,3.7,5.4
2019-10-04 12:30:00,C,1,2.7,5.8,18.4
...,...,...,...,...,...
2007-07-06 12:30:00,A,1,2.0,7.0,14.0
2007-06-01 12:30:00,A,1,4.0,9.0,20.0
2007-05-04 12:30:00,A,1,5.0,8.0,15.0


In [65]:
def df_escenarios():
    

,time,open,high,low,close,tick_volume,spread,real_volume
26,2009-02-06 13:24:00,1.1709,1.1709,1.1707,1.1708,8,40,0
27,2009-02-06 13:25:00,1.1709,1.1710,1.1708,1.1710,4,40,0
28,2009-02-06 13:26:00,1.1709,1.1710,1.1708,1.1708,10,40,0
29,2009-02-06 13:27:00,1.1709,1.1710,1.1708,1.1709,9,40,0
30,2009-02-06 13:28:00,1.1710,1.1710,1.1709,1.1710,5,40,0
31,2009-02-06 13:31:00,1.1711,1.1711,1.1709,1.1709,3,40,0
32,2009-02-06 13:32:00,1.1710,1.1720,1.1710,1.1720,12,40,0
33,2009-02-06 13:33:00,1.1719,1.1720,1.1717,1.1717,15,40,0
34,2009-02-06 13:34:00,1.1718,1.1718,1.1714,1.1716,13,40,0
35,2009-02-06 13:35:00,1.1715,1.1717,1.1715,1.1716,10,40,0


In [60]:
dates[121][0], dates[121][-1]

TypeError: 'Timestamp' object is not subscriptable

In [18]:
prices = fnmt5.f_hist_prices(mt5_client, [asset], 'M1', dates[121][0], dates[121][-1]).get(asset)

In [19]:
prices

,time,open,high,low,close,tick_volume,spread,real_volume
0,1262980740,1.02255,1.02266,1.0225,1.02265,28,24,0
